> Задача 1. Реализация метода распределения экспериментов по бакетам
  

Реализуйте метод для распределения экспериментов по бакетам.

Реализуйте метод add_experiment класса SplittingService.

In [ ]:
from pydantic import BaseModel


class Experiment(BaseModel):
    """
    id - идентификатор эксперимента.
    buckets_count - необходимое количество бакетов.
    conflicts - список идентификаторов экспериментов, которые нельзя проводить
        одновременно на одних и тех же пользователях.
    """
    id: int
    buckets_count: int
    conflicts: list[int] = []


class SplittingService:

    def __init__(self, buckets_count):
        """Класс для распределения экспериментов и пользователей по бакетам.

        :param buckets_count (int): количество бакетов.
        """
        self.buckets_count = buckets_count
        self.buckets = [[] for _ in range(buckets_count)]

    def add_experiment(self, experiment):
        """Проверяет можно ли добавить эксперимент, добавляет если можно.

        :param experiment (Experiment): параметры эксперимента, который нужно запустить
        :return success, buckets:
            success (boolean) - можно ли добавить эксперимент, True - можно, иначе - False
            buckets (list[list[int]]]) - список бакетов, в каждом бакете перечислены идентификаторы эксперименты,
                которые в нём проводятся.
        """
        # список из элементов [bucket_id, количество совместных экспериментов]
        available_buckets_meta = []
        for bucket_id, bucket in enumerate(self.buckets):
            if set(experiment.conflicts) & set(bucket):
                continue
            available_buckets_meta.append((bucket_id, len(bucket)))
        if len(available_buckets_meta) < experiment.buckets_count:
            return False, self.buckets
        sorted_available_buckets_meta = sorted(available_buckets_meta, key=lambda x: -x[1])
        for bucket_id, _ in sorted_available_buckets_meta[:experiment.buckets_count]:
            self.buckets[bucket_id].append(experiment.id)
        return True, self.buckets


def check_correct_buckets(buckets, experiments):
    for experiment in experiments:
        buckets_with_exp = [b for b in buckets if experiment.id in b]
        assert experiment.buckets_count == len(buckets_with_exp), 'Неверное количество бакетов с экспериментом'
        parallel_experiments = set(sum(buckets_with_exp, []))
        err_msg = 'Несовместные эксперименты в одном бакете'
        assert len(set(experiment.conflicts) & parallel_experiments) == 0, err_msg


if __name__ == '__main__':
    experiments = [
        Experiment(id=1, buckets_count=4, conflicts=[4]),
        Experiment(id=2, buckets_count=2, conflicts=[3]),
        Experiment(id=3, buckets_count=2, conflicts=[2]),
        Experiment(id=4, buckets_count=1, conflicts=[1]),
    ]
    ideal_answers = [True, True, True, False]

    splitting_service = SplittingService(buckets_count=4)
    added_experiments = []
    for index, (experiment, ideal_answer) in enumerate(zip(experiments, ideal_answers)):
        success, buckets = splitting_service.add_experiment(experiment)
        assert success == ideal_answer, 'Сплит-система работает неоптимально или некорректно.'
        if success:
            added_experiments.append(experiment)
        check_correct_buckets(buckets, added_experiments)
    print('simple test passed')


> Задача 2. Реализация метода распределения пользователей по бакетам
  

Реализуйте метод для распределения пользователей по бакетам и группам.

Реализуйте метод process_user класса SplittingService.

In [ ]:
from pydantic import BaseModel
import hashlib

class Experiment(BaseModel):
    """
    id - идентификатор эксперимента.
    salt - соль эксперимента (для случайного распределения пользователей на контрольную/пилотную группы)
    """
    id: int
    salt: str


class SplittingService:

    def __init__(self, buckets_count, bucket_salt, buckets=None, id2experiment=None):
        """Класс для распределения экспериментов и пользователей по бакетам.

        :param buckets_count (int): количество бакетов.
        :param bucket_salt (str): соль для разбиения пользователей по бакетам.
            При одной соли каждый пользователь должен всегда попадать в один и тот же бакет.
            Если изменить соль, то распределение людей по бакетам должно измениться.
        :param buckets (list[list[int]]) - список бакетов, в каждом бакете перечислены идентификаторы
            эксперименты, которые в нём проводятся.
        :param id2experiment (dict[int, Experiment]) - словарь пар: идентификатор эксперимента - эксперимент.
        """
        self.buckets_count = buckets_count
        self.bucket_salt = bucket_salt
        if buckets:
            self.buckets = buckets
        else:
            self.buckets = [[] for _ in range(buckets_count)]
        if id2experiment:
            self.id2experiment = id2experiment
        else:
            self.id2experiment = {}

    def process_user(self, user_id):
        """Определяет в какие эксперименты попадает пользователь.

        Сначала нужно определить бакет пользователя.
        Затем для каждого эксперимента в этом бакете выбрать пилотную или контрольную группу.

        :param user_id (str): идентификатор пользователя
        :return bucket_id, experiment_groups:
            - bucket_id (int) - номер бакета (индекс элемента в self.buckets)
            - experiment_groups (list[tuple]) - список пар: id эксперимента, группа.
                Группы: 'A', 'B'.
            Пример: (8, [(194, 'A'), (73, 'B')])
        """
        # YOUR_CODE_HERE
        # Определение номера бакета пользователя
        hashed_user_id = int(hashlib.md5((user_id + self.bucket_salt).encode()).hexdigest(), 16)
        bucket_id = hashed_user_id % self.buckets_count

        # Определение групп для каждого эксперимента в бакете с использованием второго хеша
        experiment_groups = []
        for exp_id in self.buckets[bucket_id]:
            experiment = self.id2experiment[exp_id]
            hashed_exp_user_id = int(hashlib.md5((str(exp_id) + experiment.salt + user_id).encode()).hexdigest(), 16)
            group = 'A' if hashed_exp_user_id % 2 == 0 else 'B'
            experiment_groups.append((exp_id, group))

        return bucket_id, experiment_groups


if __name__ == '__main__':
    id2experiment = {
        0: Experiment(id=0, salt='0'),
        1: Experiment(id=1, salt='1')
    }
    buckets = [[0, 1], [1], [], []]
    buckets_count = len(buckets)
    bucket_salt = 'a2N4'

    splitting_service = SplittingService(buckets_count, bucket_salt, buckets, id2experiment)
    user_ids = [str(x) for x in range(1000)]
    for user_id in user_ids:
        bucket_id, experiment_groups = splitting_service.process_user(user_id)
        assert bucket_id in [0, 1, 2, 3], 'Неверный bucket_id'
        assert len(experiment_groups) == len(buckets[bucket_id]), 'Неверное количество экспериментов в бакете'
        for exp_id, group in experiment_groups:
            assert exp_id in id2experiment, 'Неверный experiment_id'
            assert group in ['A', 'B'], 'Неверная group'
    print('simple test passed')
